## Phase frequency trend 

In [ ]:
!pwd

In [ ]:
import numpy as np
import scipy as sc
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from scipy import stats
from scipy import signal

In [ ]:
def theta1_cal( pc1, pc2 ):
    theta1           =  np.zeros(pc2.size)
    angle180         =  np.arctan2(-pc2, -pc1) * 180 / np.pi
    indNeg           =  np.where(angle180<0)[0]
    angle360         = angle180
    angle360[indNeg] = angle180[indNeg] + 360
    theta1           = np.around( ( angle360 + 22.5 ) * 8 / 360 )
    return theta1

def seas_separate(theta1_rmm,amp1_rmm,s_year,e_year):
    ph1=theta1_rmm.loc[str(s_year)+'-11-01':str(e_year)+'-04-30']*1
    a1=amp1_rmm.loc[str(s_year)+'-11-01':str(e_year)+'-04-30']*1
    return ph1,a1


def phase_freq_cal(theta1_rmm,amp1_rmm,threshold,s_year,n):
    t123 = np.zeros((n,))
    t567 = np.zeros((n,))
    t78 =np.zeros((n,))
    n1   = s_year
    
    for i in np.arange(n):
        a=[];p=[];
        p,a  = seas_separate(theta1_rmm,amp1_rmm,n1,n1+1)
        k    = np.where(a<threshold)[0]
        p[k] = 0  
        t123[i] = np.size(np.where(p==1)[0])+np.size(np.where(p==2)[0])+np.size(np.where(p==3)[0])
        t567[i] = np.size(np.where(p==4)[0])+np.size(np.where(p==5)[0])+np.size(np.where(p==6)[0])
        t78[i] = np.size(np.where(p==7)[0])+np.size(np.where(p==8)[0])

        n1 = n1+1
        
    return t123,t567,t78

def tot_phase(theta1_rmm,amp1_rmm,threshold,s_year,n):
    tt = np.zeros((n,))
    n1   = s_year
    for i in np.arange(n):
        a=[];p=[];
        p,a  = seas_separate(theta1_rmm,amp1_rmm,n1,n1+1)
        k    = np.where(a<threshold)[0]
        p[k] = 0  
        tt[i] = len(np.where(p!=0)[0])
        n1= n1+1
    return tt


def lowpass_scipy(signal,sample_freq,time_period,keep_mean):
    import numpy as np
    import scipy as sc
    from scipy import fftpack
    
    lowpass_signal=np.zeros(signal.shape)
    if any(np.isnan(signal)):
        raise ValueError('There is NaN in the signal')
    else:
        hf = 1./time_period

        temp_fft = sc.fftpack.fft(signal)

        fftfreq = np.fft.fftfreq(len(signal),sample_freq) ### daily data it is 1./365 ## monthly data 1./12 ## yearly data=1
          
        i1 = np.abs(fftfreq) >= hf  
        
        temp_fft[i1] = 0
        if not(keep_mean):
            temp_fft[0]=0
        lowpass_signal= np.real_if_close(sc.fftpack.ifft(temp_fft))
    
    return lowpass_signal

def lowpass_scipy_butter(signal1,wn,lt):
    from scipy import signal
    w = 2/lt # Normalize the frequency
    b, a = signal.butter(wn, w, 'low')
    lowpass_signal = signal.filtfilt(b, a, signal1)

    return lowpass_signal


def smooth(x,window_len=11,window='hanning'):
    ##https://scipy-cookbook.readthedocs.io/items/SignalSmooth.html
    """smooth the data using a window with requested size.
    
    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal 
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.
    
    input:
        x: the input signal 
        window_len: the dimension of the smoothing window; should be an odd integer
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.
    output:
        the smoothed signal
        
    example:
    t=linspace(-2,2,0.1)
    x=sin(t)+randn(len(t))*0.1
    y=smooth(x)
    
    see also: 
    
    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter
 
    TODO: the window parameter could be the window itself if an array instead of a string
    NOTE: length(output) != length(input), to correct this: return y[(window_len/2-1):-(window_len/2)] instead of just y.
    """

    if x.ndim != 1:
        raise ValueError("smooth only accepts 1 dimension arrays.")

    if x.size < window_len:
        raise ValueError("Input vector needs to be bigger than window size.")


    if window_len<3:
        return x


    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError("Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")


    s=np.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
    if window == 'flat': #moving average
        w=np.ones(window_len,'d')
    else:
        w=eval('np.'+window+'(window_len)')

    y=np.convolve(w/w.sum(),s,mode='valid')
    return y

In [ ]:
st = 1979 ; en = 2015 #; threshold = 1.0 ;
n = en-st;    
lf = 10;wn = 3
factor = 1.5

## WH04 RMM index

In [ ]:
rmm = xr.open_dataset('RMM_WH04_index_new.nc')
rmm1 = rmm['PC1'].loc[str(st)+'-01-01':str(en)+'-12-31']
rmm2 = rmm['PC2'].loc[str(st)+'-01-01':str(en)+'-12-31']

amp1_wh04 = rmm['MJO_INDEX']
phase1_wh04 = theta1_cal( rmm1, rmm2 )



In [ ]:
# rm11 = np.real(lowpass_scipy(rmm1.values,sample_freq=1,time_period=10,keep_mean=1))
rm11 = lowpass_scipy_butter(rmm1.values,wn,lf)
rmm11 = pd.Series(rm11,index  = rmm1.time.values)

# rm22 = np.real(lowpass_scipy(rmm2.values,sample_freq=1,time_period=10,keep_mean=1))
rm22 = lowpass_scipy_butter(rmm2.values,wn,lf)
rmm22 = pd.Series(rm22,index  = rmm2.time.values)


amp1_wh04_filt = (rmm11**2+rmm22**2)**0.5
phase1_wh04_filt = theta1_cal( rmm11, rmm22 )

print(np.mean(amp1_wh04_filt),np.std(amp1_wh04_filt)*factor,amp1_wh04_filt.loc['1979-01-01':'2008-12-31'].std()*factor)

In [ ]:
threshold = np.std(amp1_wh04_filt.loc['1979-01-01':'2008-12-31'])*factor
tt_wh04 = tot_phase(phase1_wh04_filt,amp1_wh04_filt,threshold,st,n)
t123_wh04,t567_wh04,t78_wh04 = phase_freq_cal(phase1_wh04_filt,amp1_wh04_filt,threshold,st,n)

plt.figure(figsize=(15,6))
plt.plot(t567_wh04)

In [ ]:
# threshold = 1#np.std(amp1_wh04.loc['1979-01-01':'2008-12-31'])*factor
# tt_wh04 = tot_phase(phase1_wh04,amp1_wh04,threshold,st,n)
# t123_wh04,t567_wh04,t78_wh04 = phase_freq_cal(phase1_wh04,amp1_wh04,threshold,st,n)
# plt.figure(figsize=(15,6))
# plt.plot(t567_wh04)
# phase1_wh04.loc['1979-11-01':'1980-04-30'],t567_wh04

## JRA 55

In [ ]:
df_jra55 = pd.read_excel('/home/cccr/supriyo/panini/filtered_data/data/jra55_mjo_index.xlsx')
df_jra55['Datetime'] = df_jra55.year.map(str) + '-' + df_jra55.month.map(str) + '-'+df_jra55.day.map(str) 
df_jra55.index = pd.to_datetime(df_jra55.Datetime)
df_jra55 = df_jra55.drop('Datetime',axis=1)

df_jra55['rmm1'] = df_jra55[['rmm1']]/df_jra55['rmm1'].loc['1979-01-01':'2008-12-31'].std()
df_jra55['rmm2'] = df_jra55[['rmm2']]/df_jra55['rmm2'].loc['1979-01-01':'2008-12-31'].std()


d1_jra55 = df_jra55['rmm1'][(df_jra55.index >= datetime.datetime(st,1,1)) & (df_jra55.index  <= datetime.datetime(en,12,31))]
d2_jra55 = df_jra55['rmm2'][(df_jra55.index >= datetime.datetime(st,1,1)) & (df_jra55.index  <= datetime.datetime(en,12,31))]

mjo_ind_jra = (d1_jra55**2+d2_jra55**2)**0.5


phase1_jra = theta1_cal( d1_jra55, d2_jra55 )
amp1_jra = mjo_ind_jra.loc[str(st)+'-01-01':str(en)+'-12-31']

print(np.mean(amp1_jra),np.std(mjo_ind_jra)*factor, np.std(amp1_jra.loc['1979-01-01':'2008-12-31'])*factor)

In [ ]:
threshjrad = 1.0 #np.std(amp1_jra.loc['1979-01-01':'2008-12-31'])*factor
tt_jra = tot_phase(phase1_jra,amp1_jra,threshjrad,st,n)
t123_jra, t567_jra,t78_jra = phase_freq_cal(phase1_jra,amp1_jra,threshjrad,st,n)
plt.figure(figsize=(10,4))
plt.plot(tt_wh04)
plt.plot(tt_jra)
plt.title('jra')

plt.figure(figsize=(10,4))
plt.plot(t123_jra-t123_wh04,label='123')
plt.plot(t567_jra-t567_wh04,label='456')
plt.plot(t78_jra-t78_wh04,label='78')
plt.bar(np.arange(36),tt_jra-tt_wh04,label='total')
plt.ylim([-40,40])
plt.legend()
plt.title('jra')

print(np.std(tt_jra-tt_wh04),np.std(t123_jra-t123_wh04),np.std(t567_jra-t567_wh04),np.std(t78_jra-t78_wh04))


In [ ]:
plt.figure(figsize=(16,8))
bins = np.arange(0,5,0.05)
n1,x1,_=plt.hist(amp1_wh04_filt,bins=bins,density=True,alpha=0.5,histtype='step',label='observed')


n2,x2,_=plt.hist(amp1_jra,bins=bins,density=True,alpha=0.5,histtype='step',label='jra')

plt.clf()
plt.figure(figsize=(8,4))

plt.legend()
bin_centers1 = 0.5*(x1[1:]+x1[:-1])
plt.plot(bin_centers1,n1,'r',label='observed')
bin_centers2 = 0.5*(x2[1:]+x2[:-1])
plt.plot(bin_centers2,n2,'g',label='jra')
plt.legend()

## Oliver Index

In [ ]:
df2 = pd.read_csv('rmm_oliver_20CRv3.csv')
df2['Datetime'] = df2.year.map(str) + '-' + df2.month.map(str) + '-'+df2.day.map(str) 
df2.index = pd.to_datetime(df2.Datetime)
df2 = df2.drop('Datetime',axis=1)
df2['IHR1'] = df2[['IHR1']]/df2['IHR1'].loc['1979-01-01':'2008-12-31'].std()
df2['IHR2'] = df2[['IHR2']]/df2['IHR2'].loc['1979-01-01':'2008-12-31'].std()


d1_ol = df2['IHR1'][(df2.index >= datetime.datetime(st,1,1)) & (df2.index  <= datetime.datetime(en,12,31))]
d2_ol = df2['IHR2'][(df2.index >= datetime.datetime(st,1,1)) & (df2.index  <= datetime.datetime(en,12,31))]

mjo_ind_ol = (d1_ol**2+d2_ol**2)**0.5


phase1_ol = theta1_cal( d1_ol, d2_ol )
amp1_ol = mjo_ind_ol.loc[str(st)+'-01-01':str(en)+'-12-31']

print(np.mean(amp1_ol),np.std(amp1_ol)*factor, np.std(amp1_ol.loc['1979-01-01':'2008-12-31'])*factor)

In [ ]:
threshold = np.std(amp1_ol.loc['1979-01-01':'2008-12-31'])*factor
tt_ol = tot_phase(phase1_ol,amp1_ol,threshold,st,n)
t123_ol, t567_ol,t78_ol = phase_freq_cal(phase1_ol,amp1_ol,threshold,st,n)
plt.figure(figsize=(10,4))
plt.plot(tt_wh04)
plt.plot(tt_ol)
plt.title('OL')

plt.figure(figsize=(10,4))
plt.plot(t123_ol-t123_wh04,label='123')
plt.plot(t567_ol-t567_wh04,label='456')
plt.plot(t78_ol-t78_wh04,label='78')
plt.bar(np.arange(36),tt_ol-tt_wh04,label='total')
plt.ylim([-40,40])
plt.legend()
plt.title('OL')

print(np.std(tt_ol-tt_wh04),np.std(t123_ol-t123_wh04),np.std(t567_ol-t567_wh04),np.std(t78_ol-t78_wh04))


In [ ]:
plt.figure(figsize=(16,8))
bins = np.arange(0,5,0.05)
n1,x1,_=plt.hist(amp1_wh04_filt,bins=bins,density=True,alpha=0.5,histtype='step',label='observed')


n2,x2,_=plt.hist(amp1_ol,bins=bins,density=True,alpha=0.5,histtype='step',label='ol')

plt.clf()
plt.figure(figsize=(8,4))

plt.legend()
bin_centers1 = 0.5*(x1[1:]+x1[:-1])
plt.plot(bin_centers1,n1,'r',label='observed')
bin_centers2 = 0.5*(x2[1:]+x2[:-1])
plt.plot(bin_centers2,n2,'g',label='ol')
plt.legend()

## Convolution  1D

In [ ]:
df11 = pd.read_csv('/home/cccr/supriyo/panini/filtered_data/historical/JGRJD/conv1d/ensamble4/ensamble_conv1d_jgrjd_rmm1_p1.csv')

df11.columns = ['Datetime','rmm1']
df11.index = pd.to_datetime(df11.Datetime)
df11_conv = df11.drop('Datetime',axis=1)



df22 = pd.read_csv('/home/cccr/supriyo/panini/filtered_data/historical/JGRJD/conv1d/ensamble4/ensamble_conv1d_jgrjd_rmm2.csv')

df22.columns = ['Datetime','rmm2']
df22.index = pd.to_datetime(df22.Datetime)
df22_conv = df22.drop('Datetime',axis=1)


d1_conv = df11_conv['rmm1'][(df11_conv.index >= datetime.datetime(st,1,1)) & (df11_conv.index  <= datetime.datetime(en,12,31))]
d2_conv = df22_conv['rmm2'][(df22_conv.index >= datetime.datetime(st,1,1)) & (df22_conv.index  <= datetime.datetime(en,12,31))]

mjo_ind_conv = (d1_conv**2+d2_conv**2)**0.5

phase1_conv = theta1_cal( d1_conv, d2_conv )
amp1_conv = mjo_ind_conv.loc[str(st)+'-01-01':str(en)+'-12-31']

print(np.mean(amp1_conv),np.std(amp1_conv)*factor,np.std(amp1_conv.loc['1979-01-01':'2008-12-31'])*factor)

In [ ]:
threshold = np.std(amp1_conv.loc['1979-01-01':'2008-12-31'])*factor
tt_conv = tot_phase(phase1_conv,amp1_conv,threshold,st,n)
t123_conv, t567_conv,t78_conv = phase_freq_cal(phase1_conv,amp1_conv,threshold,st,n)
plt.figure(figsize=(10,4))
plt.plot(tt_wh04)
plt.plot(tt_conv)
plt.title('CONV')

plt.figure(figsize=(10,4))
plt.plot(t123_conv-t123_wh04,label='123')
plt.plot(t567_conv-t567_wh04,label='456')
plt.plot(t78_conv-t78_wh04,label='78')
plt.bar(np.arange(36),tt_conv-tt_wh04,label='total')
plt.ylim([-40,40])
plt.title('CONV')
plt.legend()
np.std(tt_conv-tt_wh04),np.std(t123_conv-t123_wh04),np.std(t567_conv-t567_wh04),np.std(t78_conv-t78_wh04)

In [ ]:
plt.figure(figsize=(16,8))
bins = np.arange(0,5,0.05)

n1,x1,_=plt.hist(amp1_wh04_filt,bins=bins,density=True,alpha=0.5,histtype='step',label='observed')

n2,x2,_=plt.hist(amp1_conv,bins=bins,density=True,alpha=0.5,histtype='step',label='conv')

plt.clf()
plt.figure(figsize=(8,4))

plt.legend()
bin_centers1 = 0.5*(x1[1:]+x1[:-1])
plt.plot(bin_centers1,n1,'r',label='observed')
bin_centers2 = 0.5*(x2[1:]+x2[:-1])
plt.plot(bin_centers2,n2,'g',label='conv')
# bin_centers3 = 0.5*(x3[1:]+x3[:-1])
# plt.plot(bin_centers3,n3,'b',label='oliver')
plt.legend()

## convolution 2D

In [ ]:
df11 = pd.read_csv('/home/cccr/supriyo/panini/filtered_data/historical/JGRJD/deep_CNN/rmm1_conv_minmaxscale_1905_2015.csv')

df11.columns = ['Datetime','rmm1']
df11.index = pd.to_datetime(df11.Datetime)
df11_c2d = df11.drop('Datetime',axis=1)
df11_c2d = df11_c2d[['rmm1']]/df11_c2d['rmm1'].loc['1979-01-01':'2008-12-31'].std()



df22 = pd.read_csv('/home/cccr/supriyo/panini/filtered_data/historical/JGRJD/deep_CNN/rmm2_conv_minmaxscale_1905_2015.csv')

df22.columns = ['Datetime','rmm2']
df22.index = pd.to_datetime(df22.Datetime)
df22_c2d = df22.drop('Datetime',axis=1)
df22_c2d = df22_c2d[['rmm2']]/df22_c2d['rmm2'].loc['1979-01-01':'2008-12-31'].std()


d1_c2d = df11_c2d['rmm1'][(df11_c2d.index >= datetime.datetime(st,1,1)) & (df11_c2d.index  <= datetime.datetime(en,12,31))]
d2_c2d = df22_c2d['rmm2'][(df22_c2d.index >= datetime.datetime(st,1,1)) & (df22_c2d.index  <= datetime.datetime(en,12,31))]

mjo_ind_c2d = (d1_c2d**2+d2_c2d**2)**0.5

phase1_c2d = theta1_cal( d1_c2d, d2_c2d )
amp1_c2d = mjo_ind_c2d.loc[str(st)+'-01-01':str(en)+'-12-31']

print(np.mean(amp1_c2d),np.std(amp1_c2d)*factor,np.std(amp1_c2d.loc['1979-01-01':'2008-12-31'])*factor)

In [ ]:
threshold = np.std(amp1_c2d.loc['1979-01-01':'2008-12-31'])*factor
tt_c2d = tot_phase(phase1_c2d,amp1_c2d,threshold,st,n)
t123_c2d, t567_c2d,t78_c2d = phase_freq_cal(phase1_c2d,amp1_c2d,threshold,st,n)
plt.figure(figsize=(10,4))
plt.plot(tt_wh04)
plt.plot(tt_c2d)
plt.title('c2d')

plt.figure(figsize=(10,4))
plt.plot(t123_c2d-t123_wh04,label='123')
plt.plot(t567_c2d-t567_wh04,label='456')
plt.plot(t78_c2d-t78_wh04,label='78')
plt.bar(np.arange(36),tt_c2d-tt_wh04,label='total')
plt.ylim([-40,40])
plt.title('c2d')
plt.legend()
np.std(tt_c2d-tt_wh04),np.std(t123_c2d-t123_wh04),np.std(t567_c2d-t567_wh04),np.std(t78_c2d-t78_wh04)

In [ ]:
plt.figure(figsize=(16,8))
bins = np.arange(0,5,0.05)

n1,x1,_=plt.hist(amp1_wh04_filt,bins=bins,density=True,alpha=0.5,histtype='step',label='observed')

n2,x2,_=plt.hist(amp1_c2d,bins=bins,density=True,alpha=0.5,histtype='step',label='c2d')

plt.clf()
plt.figure(figsize=(8,4))

plt.legend()
bin_centers1 = 0.5*(x1[1:]+x1[:-1])
plt.plot(bin_centers1,n1,'r',label='observed')
bin_centers2 = 0.5*(x2[1:]+x2[:-1])
plt.plot(bin_centers2,n2,'g',label='c2d')
# bin_centers3 = 0.5*(x3[1:]+x3[:-1])
# plt.plot(bin_centers3,n3,'b',label='oliver')
plt.legend()

## MLR :

In [ ]:

df11 = pd.read_csv('/home/cccr/supriyo/panini/filtered_data/historical/JGRJD/multiple_regression/rmm1_MLR_org.txt')

df11.columns = ['Datetime','rmm1']
df11.index = pd.to_datetime(df11.Datetime)
df11 = df11.drop('Datetime',axis=1)
df11 = df11[['rmm1']]/df11['rmm1'].loc['1979-01-01':'2008-12-31'].std()


df22 = pd.read_csv('/home/cccr/supriyo/panini/filtered_data/historical/JGRJD/multiple_regression/rmm2_MLR_org.txt')
df22.columns = ['Datetime','rmm2']
df22.index = pd.to_datetime(df22.Datetime)
df22 = df22.drop('Datetime',axis=1)
df22 = df22[['rmm2']]/df22['rmm2'].loc['1979-01-01':'2008-12-31'].std()

# df11.columns =['rmm1']
# df22.columns =['rmm2']



d1_mlr = df11['rmm1'][(df11.index >= datetime.datetime(st,1,1)) & (df11.index  <= datetime.datetime(en,12,31))]
d2_mlr = df22['rmm2'][(df22.index >= datetime.datetime(st,1,1)) & (df22.index  <= datetime.datetime(en,12,31))]

mjo_ind_mlr = (d1_mlr**2+d2_mlr**2)**0.5

phase1_mlr = theta1_cal( d1_mlr, d2_mlr )
amp1_mlr = mjo_ind_mlr.loc[str(st)+'-01-01':str(en)+'-12-31']
print(np.mean(amp1_mlr),np.std(amp1_mlr)*factor,np.std(amp1_mlr.loc['1979-01-01':'2008-12-31'])*factor)

In [ ]:
threshold = np.std(amp1_mlr.loc['1979-01-01':'2008-12-31'])*factor
tt_mlr = tot_phase(phase1_mlr,amp1_mlr,threshold,st,n)
t123_mlr, t567_mlr,t78_mlr = phase_freq_cal(phase1_mlr,amp1_mlr,threshold,st,n)
plt.figure(figsize=(10,4))
plt.plot(tt_wh04)
plt.plot(tt_mlr)
plt.title('MLR')

plt.figure(figsize=(10,4))
plt.plot(t123_mlr-t123_wh04,label='123')
plt.plot(t567_mlr-t567_wh04,label='456')
plt.plot(t78_mlr-t78_wh04,label='78')
plt.bar(np.arange(36),tt_mlr-tt_wh04,label='total')
plt.ylim([-40,40])
plt.title('MLR')
plt.legend()
np.std(tt_mlr-tt_wh04),np.std(t123_mlr-t123_wh04),np.std(t567_mlr-t567_wh04),np.std(t78_mlr-t78_wh04)

In [ ]:
plt.figure(figsize=(16,8))
bins = np.arange(0,5,0.05)
n1,x1,_=plt.hist(amp1_wh04_filt,bins=bins,density=True,alpha=0.5,histtype='step',label='observed')

n2,x2,_=plt.hist(amp1_mlr,bins=bins,density=True,alpha=0.5,histtype='step',label='mlr')

plt.clf()
plt.figure(figsize=(8,4))

plt.legend()
bin_centers1 = 0.5*(x1[1:]+x1[:-1])
plt.plot(bin_centers1,n1,'r',label='observed')
bin_centers2 = 0.5*(x2[1:]+x2[:-1])
plt.plot(bin_centers2,n2,'g',label='mlr')
# bin_centers3 = 0.5*(x3[1:]+x3[:-1])
# plt.plot(bin_centers3,n3,'b',label='oliver')
plt.legend()

## SVR

In [ ]:
df_svr11 = pd.read_csv('/home/cccr/supriyo/panini/filtered_data/historical/JGRJD/SVR/rmm1_1905_2015_svr_org_new.txt')

df_svr11.columns = ['Datetime','rmm1']
df_svr11.index = pd.to_datetime(df_svr11.Datetime)
df_svr11 = df_svr11.drop('Datetime',axis=1)
df_svr11 = df_svr11[['rmm1']]/df_svr11['rmm1'].loc['1979-01-01':'2008-12-31'].std()


df_svr22 = pd.read_csv('/home/cccr/supriyo/panini/filtered_data/historical/JGRJD/SVR/rmm2_1905_2015_svr_org_new.txt')
df_svr22.columns = ['Datetime','rmm2']
df_svr22.index = pd.to_datetime(df_svr22.Datetime)
df_svr22 = df_svr22.drop('Datetime',axis=1)
df_svr22 = df_svr22[['rmm2']]/df_svr22['rmm2'].loc['1979-01-01':'2008-12-31'].std()


d1_svr = df_svr11['rmm1'][(df_svr11.index >= datetime.datetime(st,1,1)) & (df_svr11.index  <= datetime.datetime(en,12,31))]
d2_svr = df_svr22['rmm2'][(df_svr22.index >= datetime.datetime(st,1,1)) & (df_svr22.index  <= datetime.datetime(en,12,31))]

mjo_ind_svr = (d1_svr**2+d2_svr**2)**0.5

phase1_svr = theta1_cal( d1_svr, d2_svr )
amp1_svr = mjo_ind_svr.loc[str(st)+'-01-01':str(en)+'-12-31']
print(np.mean(amp1_svr),np.std(amp1_svr)*factor,np.std(amp1_svr.loc['1979-01-01':'2008-12-31'])*factor)

In [ ]:
threshold = np.std(amp1_svr.loc['1979-01-01':'2008-12-31'])*factor
tt_svr = tot_phase(phase1_svr,amp1_svr,threshold,st,n)
t123_svr, t567_svr,t78_svr = phase_freq_cal(phase1_svr,amp1_svr,threshold,st,n)
plt.figure(figsize=(10,4))
plt.plot(tt_wh04)
plt.plot(tt_svr)
plt.title('svr')

plt.figure(figsize=(10,4))
plt.plot(t123_svr-t123_wh04,label='123')
plt.plot(t567_svr-t567_wh04,label='456')
plt.plot(t78_svr-t78_wh04,label='78')
plt.bar(np.arange(36),tt_svr-tt_wh04,label='total')
plt.ylim([-40,40])
plt.title('svr')
plt.legend()
print(np.std(tt_svr-tt_wh04),np.std(t123_svr-t123_wh04),np.std(t567_svr-t567_wh04),np.std(t78_svr-t78_wh04))

In [ ]:
plt.figure(figsize=(16,8))
bins = np.arange(0,5,0.05)

n1,x1,_=plt.hist(amp1_wh04_filt,bins=bins,density=True,alpha=0.5,histtype='step',label='observed')

n2,x2,_=plt.hist(amp1_svr,bins=bins,density=True,alpha=0.5,histtype='step',label='svr')

plt.clf()
plt.figure(figsize=(8,4))

plt.legend()
bin_centers1 = 0.5*(x1[1:]+x1[:-1])
plt.plot(bin_centers1,n1,'r',label='observed')
bin_centers2 = 0.5*(x2[1:]+x2[:-1])
plt.plot(bin_centers2,n2,'g',label='svr')
# bin_centers3 = 0.5*(x3[1:]+x3[:-1])
# plt.plot(bin_centers3,n3,'b',label='oliver')
plt.legend()

In [ ]:
plt.figure (figsize=(15,5))
plt.plot(t567_wh04,'k+--',label = 'Wh04')
m_wh04,c_wh04,r_wh04,p_wh04,_ = stats.linregress(np.arange(1,len(t567_wh04)+1),t567_wh04)
plt.plot(m_wh04*np.arange(1,len(t567_wh04)+1)+c_wh04,'k')

plt.plot(t567_jra,'c+--',label = 'jra55')
m_jra,c_jra,r_jra,p_jra,_ = stats.linregress(np.arange(1,len(t567_jra)+1),t567_jra)
plt.plot(m_jra*np.arange(1,len(t567_jra)+1)+c_jra,'--c')

plt.plot(t567_c2d,'g',label ='c2d')
m_c2d,c_c2d,r_c2d,p_c2d,_ = stats.linregress(np.arange(1,len(t567_c2d)+1),t567_c2d)
plt.plot(m_c2d*np.arange(1,len(t567_c2d)+1)+c_c2d,'--g')

plt.plot(t567_ol,'orange',label ='ol')
m_ol,c_ol,r_ol,p_ol,_ = stats.linregress(np.arange(1,len(t567_ol)+1),t567_ol)
plt.plot(m_ol*np.arange(1,len(t567_ol)+1)+c_ol,linestyle='--',color='orange')


plt.plot(t567_mlr,'r',label ='mlr')
m_mlr,c_mlr,r_mlr,p_mlr,_ = stats.linregress(np.arange(1,len(t567_mlr)+1),t567_mlr)
plt.plot(m_mlr*np.arange(1,len(t567_mlr)+1)+c_mlr,'--r')


plt.plot(t567_conv,'b',label = 'conv')
m_conv,c_conv,r_conv,p_conv,_ = stats.linregress(np.arange(1,len(t567_conv)+1),t567_conv)
plt.plot(m_conv*np.arange(1,len(t567_conv)+1)+c_conv,'--b')
plt.legend()
plt.title('phase 456 trend')

list = np.arange(1979,2016,3).tolist()
s = [str(i) for i in list] 
plt.xticks(np.arange(0,40,3),s)
# ax[0].set_xticklabels(s)
m_wh04,m_jra,m_conv,m_mlr,m_c2d,p_wh04,p_jra,p_conv,p_mlr,r_wh04,p_c2d,r_conv,r_mlr
# plt.plot(t567_conv_hist[-36:],'cyan')

In [ ]:
window_len = 3 

t567_conv_3yr= smooth(t567_conv,window_len,window='flat')[1:-1]
t567_ol_3yr= smooth(t567_ol,window_len,window='flat')[1:-1]
t567_wh04_3yr= smooth(t567_wh04,window_len,window='flat')[1:-1]
t567_mlr_3yr= smooth(t567_mlr,window_len,window='flat')[1:-1]
t567_svr_3yr= smooth(t567_svr,window_len,window='flat')[1:-1]
t567_jra_3yr= smooth(t567_jra,window_len,window='flat')[1:-1]
t567_c2d_3yr= smooth(t567_c2d,window_len,window='flat')[1:-1]

t123_conv_3yr= smooth(t123_conv,window_len,window='flat')[1:-1]
t123_ol_3yr= smooth(t123_ol,window_len,window='flat')[1:-1]
t123_wh04_3yr= smooth(t123_wh04,window_len,window='flat')[1:-1]
t123_mlr_3yr= smooth(t123_mlr,window_len,window='flat')[1:-1]
t123_svr_3yr= smooth(t123_svr,window_len,window='flat')[1:-1]
t123_jra_3yr= smooth(t123_jra,window_len,window='flat')[1:-1]
t123_c2d_3yr= smooth(t123_c2d,window_len,window='flat')[1:-1]


fig,ax = plt.subplots(2,1,figsize=(15,10))
fig.tight_layout(pad=2)

SMALL_SIZE = 10
MEDIUM_SIZE = 12#10
BIGGER_SIZE = 14

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title



ax[0].plot(t567_conv_3yr,'o-',label = 'convolution1D: m=0.22,p=0.02')
m_conv,c_conv,r_conv,p_conv,_ = stats.linregress(np.arange(1,len(t567_conv)+1),t567_conv_3yr)
ax[0].plot(m_conv*np.arange(1,len(t567_conv)+1)+c_conv,'--C0')

ax[0].plot(t567_ol_3yr,'-o',color='orange',label ='OT12: m=0.13,p=0.32')
m_ol,c_ol,r_ol,p_ol,_ = stats.linregress(np.arange(1,len(t567_ol)+1),t567_ol_3yr)
ax[0].plot(m_ol*np.arange(1,len(t567_ol)+1)+c_ol,color='orange',linestyle='--')


ax[0].plot(t567_wh04_3yr,'ko-',label = 'WH04: m=0.43,p=0.001')
m_wh04,c_wh04,r_wh04,p_wh04,_ = stats.linregress(np.arange(1,len(t567_wh04)+1),t567_wh04_3yr)
ax[0].plot(m_wh04*np.arange(1,len(t567_wh04)+1)+c_wh04,'--k')

ax[0].plot(t567_mlr_3yr,'ro-',label ='MLR: m=0.22,p=0.08')
m_mlr,c_mlr,r_mlr,p_mlr,_ = stats.linregress(np.arange(1,len(t567_mlr)+1),t567_mlr_3yr)
ax[0].plot(m_mlr*np.arange(1,len(t567_mlr)+1)+c_mlr,'--r')


ax[0].plot(t567_svr_3yr,'go-',label ='SVR: m=0.22,p=0.08')
m_svr,c_svr,r_svr,p_svr,_ = stats.linregress(np.arange(1,len(t567_svr)+1),t567_svr_3yr)
ax[0].plot(m_svr*np.arange(1,len(t567_svr)+1)+c_svr,'--g')

ax[0].plot(t567_jra_3yr,'co-',label ='JRA55: m=0.22,p=0.08')
m_jra,c_jra,r_jra,p_jra,_ = stats.linregress(np.arange(1,len(t567_jra)+1),t567_jra_3yr)
ax[0].plot(m_jra*np.arange(1,len(t567_jra)+1)+c_jra,'--c')


ax[0].plot(t567_c2d_3yr,'magenta',marker='o',label ='C2D: m=0.22,p=0.08')
m_c2d,c_c2d,r_c2d,p_c2d,_ = stats.linregress(np.arange(1,len(t567_c2d)+1),t567_c2d_3yr)
ax[0].plot(m_c2d*np.arange(1,len(t567_c2d)+1)+c_c2d,'--',color = 'magenta')

list = np.arange(1979,2016,3).tolist()
s = [str(i) for i in list] 
ax[0].set_xticks(np.arange(0,40,3))
ax[0].set_xticklabels(s)


ax[0].set_title('phase 4,5,6')
ax[0].legend(bbox_to_anchor=(1,0),loc="lower right",ncol=4)
ax[0].set_ylim([20,65])

print(m_conv,m_ol,m_wh04,m_mlr,m_svr,m_c2d,m_jra)
print(p_conv,p_ol,m_wh04,p_mlr,p_svr,p_c2d,p_jra)
r_wh04,r_conv,r_mlr


######################################################################################

ax[1].plot(t123_conv_3yr,'o-',label = 'convolution1D: m=-0.18,p=0.21')
m_conv,c_conv,r_conv,p_conv,_ = stats.linregress(np.arange(1,len(t123_conv)+1),t123_conv_3yr)
ax[1].plot(m_conv*np.arange(1,len(t123_conv)+1)+c_conv,'--C0')

ax[1].plot(t123_ol_3yr,'-o',color='orange',label ='OT12: m=-0.14,p=0.24')
m_ol,c_ol,r_ol,p_ol,_ = stats.linregress(np.arange(1,len(t123_ol)+1),t123_ol_3yr)
ax[1].plot(m_ol*np.arange(1,len(t123_ol)+1)+c_ol,color='orange',linestyle='--')


ax[1].plot(t123_wh04_3yr,'ko-',label = 'WH04: m=-0.25,p=0.08')
m_wh04,c_wh04,r_wh04,p_wh04,_ = stats.linregress(np.arange(1,len(t123_wh04)+1),t123_wh04_3yr)
ax[1].plot(m_wh04*np.arange(1,len(t123_wh04)+1)+c_wh04,'--k')

ax[1].plot(t123_mlr_3yr,'ro-',label ='MLR: m=-0.15,p=0.28')
m_mlr,c_mlr,r_mlr,p_mlr,_ = stats.linregress(np.arange(1,len(t123_mlr)+1),t123_mlr_3yr)
ax[1].plot(m_mlr*np.arange(1,len(t123_mlr)+1)+c_mlr,'--r')


ax[1].plot(t123_svr_3yr,'go-',label ='SVR: m=-0.17,p=0.21')
m_svr,c_svr,r_svr,p_svr,_ = stats.linregress(np.arange(1,len(t123_svr)+1),t123_svr_3yr)
ax[1].plot(m_svr*np.arange(1,len(t123_svr)+1)+c_svr,'--g')


ax[1].plot(t123_jra_3yr,'co-',label ='JRA55: m=0.22,p=0.08')
m_jra,c_jra,r_jra,p_jra,_ = stats.linregress(np.arange(1,len(t123_jra)+1),t123_jra_3yr)
ax[1].plot(m_jra*np.arange(1,len(t123_jra)+1)+c_jra,'--c')


ax[1].plot(t123_c2d_3yr,'magenta',marker='o',label ='C2D: m=0.22,p=0.08')
m_c2d,c_c2d,r_c2d,p_c2d,_ = stats.linregress(np.arange(1,len(t123_c2d)+1),t123_c2d_3yr)
ax[1].plot(m_c2d*np.arange(1,len(t123_c2d)+1)+c_c2d,'--',color = 'magenta')

ax[1].set_xticks(np.arange(0,40,3))
ax[1].set_xticklabels(s)

# ax[1].xlim([0,36])

ax[1].set_title('phase 1,2,3')
ax[1].legend(bbox_to_anchor=(1,0),loc="lower right",ncol=4)
ax[1].set_ylim([20,65])


fig.text(0.5, -0.02, 'year', ha='center')
fig.text(0., 0.5, '3yr running phase occurences', va='center', rotation='vertical')

print(m_conv,m_ol,m_wh04,m_mlr,m_svr,m_c2d,m_jra)
print(p_conv,p_ol,m_wh04,p_mlr,p_svr,p_c2d,p_jra)



In [ ]:
plt.plot(pd.DataFrame(t567_conv).rolling(3).mean(),'b')
plt.plot(pd.DataFrame(t567_wh04).rolling(3).mean(),'k')
plt.plot(pd.DataFrame(t567_jra).rolling(3).mean(),'r')
plt.plot(pd.DataFrame(t567_c2d).rolling(3).mean(),'g')


In [ ]:
plt.plot(t567_conv_3yr-t567_wh04_3yr)

plt.plot(t567_mlr_3yr-t567_wh04_3yr)


In [ ]:
plt.figure (figsize=(15,5))
plt.plot(t123_wh04,'k+--',label = 'Wh04')
m_wh04,c_wh04,r_wh04,p_wh04,_ = stats.linregress(np.arange(1,len(t123_wh04)+1),t123_wh04)
plt.plot(m_wh04*np.arange(1,len(t123_wh04)+1)+c_wh04,'k')
plt.plot(t123_mlr,'r',label ='mlr')
m_mlr,c_mlr,r_mlr,p_mlr,_ = stats.linregress(np.arange(1,len(t123_mlr)+1),t123_mlr)
plt.plot(m_mlr*np.arange(1,len(t123_mlr)+1)+c_mlr,'--r')
plt.plot(t123_conv,'b',label = 'convmlrution1D')
m_conv,c_conv,r_conv,p_conv,_ = stats.linregress(np.arange(1,len(t123_conv)+1),t123_conv)
plt.plot(m_conv*np.arange(1,len(t123_conv)+1)+c_conv,'--b')
plt.legend()
plt.title('phase 123 trend')
m_wh04,m_conv,m_mlr,p_wh04,p_conv,p_mlr,r_wh04,r_conv,r_mlr


In [ ]:
# plt.plot(smooth(t123_conv,window_len,window='flat')[1:-1])
# plt.plot(pd.DataFrame(t123_conv).rolling(window_len,center=True).mean())

plt.plot(t123_conv_3yr-t123_wh04_3yr)
plt.plot(t123_mlr_3yr-t123_wh04_3yr)


In [ ]:
np.corrcoef(signal.detrend(t567_conv),signal.detrend(t567_wh04))[0,1],np.corrcoef(signal.detrend(t123_conv),signal.detrend(t123_wh04))[0,1]

In [ ]:
np.corrcoef(signal.detrend(t567_ol),signal.detrend(t567_wh04))[0,1],np.corrcoef(signal.detrend(t123_ol),signal.detrend(t123_wh04))[0,1]

## 1905-2015

In [ ]:
import pandas as pd
from scipy import stats
plt.rcParams['figure.figsize'] = (20,5)

factor = 1.5

st = 1905; en = 2015;
n= en-st


############################################# conv ######################################################################

d1_conv_hist = df11_conv['rmm1'][(df11_conv.index >= datetime.datetime(st,1,1)) & (df11_conv.index  <= datetime.datetime(en,12,31))]
d2_conv_hist = df22_conv['rmm2'][(df22_conv.index >= datetime.datetime(st,1,1)) & (df22_conv.index  <= datetime.datetime(en,12,31))]

mjo_ind_conv_hist = (d1_conv_hist**2+d2_conv_hist**2)**0.5

phase1_conv_hist = theta1_cal( d1_conv_hist, d2_conv_hist )
amp1_conv_hist = mjo_ind_conv_hist *1
threshold_c = np.std(amp1_conv_hist.loc['1979-01-01':'2008-12-31'])*factor
t123_conv_hist, t567_conv_hist,_ = phase_freq_cal(phase1_conv_hist,amp1_conv_hist,threshold_c,st,n)




############################################## ol ##############################################################################################

df2 = pd.read_csv('rmm_oliver_20CRv3.csv')
df2['Datetime'] = df2.year.map(str) + '-' + df2.month.map(str) + '-'+df2.day.map(str) 
df2.index = pd.to_datetime(df2.Datetime)
df2 = df2.drop('Datetime',axis=1)


d1_ol = df2['IHR1'][(df2.index >= datetime.datetime(st,1,1)) & (df2.index  <= datetime.datetime(en,12,31))]
d2_ol = df2['IHR2'][(df2.index >= datetime.datetime(st,1,1)) & (df2.index  <= datetime.datetime(en,12,31))]

mjo_ind_ol = (d1_ol**2+d2_ol**2)**0.5


phase1_ol_hist = theta1_cal( d1_ol, d2_ol )
amp1_ol_hist = mjo_ind_ol*1
threshold_o = np.std(amp1_ol.loc['1979-01-01':'2008-12-31'])*factor

t123_ol_hist, t567_ol_hist,_ = phase_freq_cal(phase1_ol_hist,amp1_ol_hist,threshold_o,st,n)

########################################### JRA 55 ###########################################################################
stj = 1958;enj=2015;nj=enj-stj
d1_jra = df_jra55['rmm1'][(df_jra55.index >= datetime.datetime(stj,1,1)) & (df_jra55.index  <= datetime.datetime(enj,12,31))]
d2_jra = df_jra55['rmm2'][(df_jra55.index >= datetime.datetime(stj,1,1)) & (df_jra55.index  <= datetime.datetime(enj,12,31))]

mjo_ind_jra = (d1_jra**2+d2_jra**2)**0.5


phase1_jra_hist = theta1_cal( d1_jra, d2_jra )
amp1_jra_hist = mjo_ind_jra.loc[str(stj)+'-01-01':str(enj)+'-12-31']
threshold_j = np.std(amp1_jra.loc['1979-01-01':'2008-12-31'])*factor

t123_jra_hist, t567_jra_hist,_ = phase_freq_cal(phase1_jra_hist,amp1_jra_hist,threshold_j,stj,nj)

################################################# mlr ###################################################################
d1_mlr = df11['rmm1'][(df11.index >= datetime.datetime(st,1,1)) & (df11.index  <= datetime.datetime(en,12,31))]
d2_mlr = df22['rmm2'][(df22.index >= datetime.datetime(st,1,1)) & (df22.index  <= datetime.datetime(en,12,31))]

mjo_ind_mlr = (d1_mlr**2+d2_mlr**2)**0.5

phase1_mlr_hist = theta1_cal( d1_mlr, d2_mlr )
amp1_mlr_hist = mjo_ind_mlr.loc[str(st)+'-01-01':str(en)+'-12-31']
threshold_mlr = np.std(amp1_ol.loc['1979-01-01':'2008-12-31'])*factor
t123_mlr_hist, t567_mlr_hist,_ = phase_freq_cal(phase1_mlr_hist,amp1_mlr_hist,threshold_mlr,st,n)


###################################################### svr #########################################################
d1_svr = df_svr11['rmm1'][(df_svr11.index >= datetime.datetime(st,1,1)) & (df_svr11.index  <= datetime.datetime(en,12,31))]
d2_svr = df_svr22['rmm2'][(df_svr22.index >= datetime.datetime(st,1,1)) & (df_svr22.index  <= datetime.datetime(en,12,31))]

mjo_ind_svr = (d1_svr**2+d2_svr**2)**0.5

phase1_svr_hist = theta1_cal( d1_svr, d2_svr )
amp1_svr_hist = mjo_ind_svr.loc[str(st)+'-01-01':str(en)+'-12-31']
threshold_svr = np.std(amp1_ol.loc['1979-01-01':'2008-12-31'])*factor
t123_svr_hist, t567_svr_hist,_ = phase_freq_cal(phase1_svr_hist,amp1_svr_hist,threshold_svr,st,n)


######################################### c2d ############################################################################
d1_c2d_hist = df11_c2d['rmm1'][(df11_c2d.index >= datetime.datetime(st,1,1)) & (df11_c2d.index  <= datetime.datetime(en,12,31))]
d2_c2d_hist = df22_c2d['rmm2'][(df22_c2d.index >= datetime.datetime(st,1,1)) & (df22_c2d.index  <= datetime.datetime(en,12,31))]

mjo_ind_c2d_hist = (d1_c2d_hist**2+d2_c2d_hist**2)**0.5

phase1_c2d_hist = theta1_cal( d1_c2d_hist, d2_c2d_hist )
amp1_c2d_hist = mjo_ind_c2d_hist *1
threshold_c = np.std(amp1_c2d_hist.loc['1979-01-01':'2008-12-31'])*factor
t123_c2d_hist, t567_c2d_hist,_ = phase_freq_cal(phase1_c2d_hist,amp1_c2d_hist,threshold_c,st,n)


In [ ]:
plt.plot(t567_c2d_hist[-57:])
plt.plot(t567_mlr_hist[-57:])
plt.plot(t567_conv_hist[-57:])
plt.plot(t567_jra_hist)
np.corrcoef(t567_conv_hist[-57:],[t567_mlr_hist[-57:],t567_c2d_hist[-57:],t567_jra_hist])
plt.figure(figsize=(5,5))


## prepare a dataframe of 3 year running mean

In [ ]:
df_conv = pd.DataFrame(t567_conv_hist,index=pd.date_range('1906-01-01',periods=110,freq='Y'),columns=['duration'])
df_ol =    pd.DataFrame(t567_ol_hist,index=pd.date_range('1906-01-01',periods=110,freq='Y'),columns=['duration'])
df_wh04 =  pd.DataFrame(t567_wh04,index=pd.date_range('1980-01-01',periods=36,freq='Y'),columns=['duration'])
df_mlr = pd.DataFrame(t567_mlr_hist,index=pd.date_range('1906-01-01',periods=110,freq='Y'),columns=['duration'])
df_svr = pd.DataFrame(t567_svr_hist,index=pd.date_range('1906-01-01',periods=110,freq='Y'),columns=['duration'])
df_c2d = pd.DataFrame(t567_c2d_hist,index=pd.date_range('1906-01-01',periods=110,freq='Y'),columns=['duration'])
df_jra = pd.DataFrame(t567_jra_hist,index=pd.date_range('1959-01-01',periods=57,freq='Y'),columns=['duration'])


df_567_all=pd.concat([df_conv,df_ol,df_wh04,df_mlr,df_svr,df_c2d,df_jra],axis=1)

In [ ]:
window_len=11 ; ww = 2*(window_len//2)
yy=str(int(1906+ww/2));ll1= 110-ww;yy2=str(int(1980+ww/2));ll2=36-ww;yy3=str(int(1959+ww/2));ll3=57-ww

### different methods #######

df_conv_3yr = pd.DataFrame(smooth(t567_conv_hist,window_len=window_len,window='flat')[ww:-ww],index=pd.date_range(yy+'-01-01',periods=ll1,freq='Y'),columns=['duration'])
df_ol_3yr = pd.DataFrame(smooth(t567_ol_hist,window_len=window_len,window='flat')[ww:-ww],index=pd.date_range(yy+'-01-01',periods=ll1,freq='Y'),columns=['duration'])
df_wh04_3yr =  pd.DataFrame(smooth(t567_wh04,window_len=window_len,window='flat')[ww:-ww],index=pd.date_range(yy2+'-01-01',periods=ll2,freq='Y'),columns=['duration'])
df_mlr_3yr = pd.DataFrame(smooth(t567_mlr_hist,window_len=window_len,window='flat')[ww:-ww],index=pd.date_range(yy+'-01-01',periods=ll1,freq='Y'),columns=['duration'])
df_svr_3yr = pd.DataFrame(smooth(t567_svr_hist,window_len=window_len,window='flat')[ww:-ww],index=pd.date_range(yy+'-01-01',periods=ll1,freq='Y'),columns=['duration'])
df_c2d_3yr = pd.DataFrame(smooth(t567_c2d_hist,window_len=window_len,window='flat')[ww:-ww],index=pd.date_range(yy+'-01-01',periods=ll1,freq='Y'),columns=['duration'])
df_jra_3yr = pd.DataFrame(smooth(t567_jra_hist,window_len=window_len,window='flat')[ww:-ww],index=pd.date_range(yy3+'-01-01',periods=ll3,freq='Y'),columns=['duration'])

df_567_all_3yr=pd.concat([df_conv_3yr,df_ol_3yr,df_wh04_3yr,df_mlr_3yr,df_svr_3yr,df_c2d_3yr,df_jra_3yr],axis=1)









## PDO index

In [ ]:
pdo = pd.read_csv('pdo_index.txt',sep ='   ',header = None)
pdo.columns = ['year','pdo']
pdo.index= pdo.year
pdo =pdo.drop('year',axis=1)
pdot = pdo.loc['1906':'2015']


##################################################################

pdo=pd.read_fwf('/home/cccr/roxy/panini/JGRJD_PHD_PART_I/DATA/indices/pdo/pdo_index.txt')
pdo_sel=pdo.set_index('YEAR').loc[1905:2015]
pdo_winter=pdo_sel[['JAN','FEB','MAR','APR','NOV','DEC']].values.ravel()
k=4
pdo_winter_mean=np.zeros((110,))
for i in np.arange(110):
    pdo_winter_mean[i]=np.mean(pdo_winter[k:k+6])
    k=k+6
    
plt.plot(pdo_winter_mean)
plt.plot(pdot.values)

### PDO index
pdo_winter_3yr = pd.DataFrame(smooth(pdo_winter_mean,window_len=window_len,window='flat')[ww:-ww],index=pd.date_range(yy+'-01-01',periods=ll1,freq='Y'),columns=['duration'])



In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from mpl_toolkits.axes_grid1 import host_subplot

fig,ax=plt.subplots(1,1,figsize=(20,5))

plt.rcParams['font.weight'] = 'normal'
plt.rcParams['font.size'] = 15
plt.rcParams['image.cmap'] = 'seismic'


ax.plot(df_567_all_3yr.iloc[:,0]-df_567_all_3yr.iloc[:,0].mean(),label='convolution1D')
ax.plot(df_567_all_3yr.iloc[:,1]-df_567_all_3yr.iloc[:,1].mean(),'-',label='OT12',alpha=0.9)
ax.plot(df_567_all_3yr.iloc[:,2]-df_567_all_3yr.iloc[:,2].mean(),'k',label='WH04',alpha=0.9)
ax.plot(df_567_all_3yr.iloc[:,3]-df_567_all_3yr.iloc[:,3].mean(),'-',label='MLR',alpha=0.9)
ax.plot(df_567_all_3yr.iloc[:,4]-df_567_all_3yr.iloc[:,4].mean(),'-',label='SVR',alpha=0.9)
# ax.plot(df_567_all_3yr.iloc[:,5]-df_567_all_3yr.iloc[:,5].mean(),'-.',label='C2D',alpha=0.5,color='magenta')
ax.plot(df_567_all_3yr.iloc[:,6]-df_567_all_3yr.iloc[:,6].mean(),'-',label='JRA55',color='cyan')




ax.set_ylim([-15,15])
ax.grid(linestyle='-', linewidth=.5,alpha=0.3)
ax.axhline(0,color='k',linestyle='--')
ax.set_xlim([datetime.datetime(1905,6,1),datetime.datetime(2015,6,1)])


fig.text(0.5, -0.04, 'Year', ha='center')
fig.text(0.09, 0.5, '3yr running JO duration at 4,5,6', va='center', rotation='vertical')
ax.axvspan(datetime.datetime(1921,6,1),datetime.datetime(1945,6,1),color = 'r',alpha = 0.05)
ax.axvspan(datetime.datetime(1945,6,1),datetime.datetime(1977,6,1),color = 'b',alpha = 0.05)
ax.axvspan(datetime.datetime(1977,6,1),datetime.datetime(1999,6,1),color = 'r',alpha = 0.05)# ax.axvspan(1998,2012,alpha = 0.1)
ax.axvspan(datetime.datetime(1999,6,1),datetime.datetime(2012,6,1),color = 'b',alpha = 0.05)

plt.legend(ncol=3)
plt.title('11 year running MJO duration at 4,5,6')
ax = plt.gca()
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=60))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
# plt.gcf().autofmt_xdate() # Rotation

ax2 = ax.twinx()


ax2.plot(pdo_winter_3yr-pdo_winter_3yr.mean(),'-.',label='PDO_index',color='k')
ax2.set_ylim([-1.5,1.5])

plt.legend()
plt.show()












In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

fig,ax=plt.subplots(1,1,figsize=(20,5))
plt.rcParams['font.weight'] = 'normal'
plt.rcParams['font.size'] = 15
plt.rcParams['image.cmap'] = 'seismic'


ax.plot(df_567_all.iloc[:,0]-df_567_all.iloc[:,0].mean(),label='convolution1D')
ax.plot(df_567_all.iloc[:,1]-df_567_all.iloc[:,1].mean(),'-.',label='OT12',alpha=0.5)
ax.plot(df_567_all.iloc[:,2]-df_567_all.iloc[:,2].mean(),'k',label='WH04',alpha=0.5)
ax.plot(df_567_all.iloc[:,3]-df_567_all.iloc[:,3].mean(),'-.',label='MLR',alpha=0.5)
ax.plot(df_567_all.iloc[:,4]-df_567_all.iloc[:,4].mean(),'-.',label='SVR',alpha=0.5)
ax.plot(df_567_all.iloc[:,5]-df_567_all.iloc[:,5].mean(),'-.',label='C2D',alpha=0.5,color='magenta')
ax.plot(df_567_all.iloc[:,6]-df_567_all.iloc[:,6].mean(),'-.',label='JRA55',color='cyan')




# ax.set_ylim([0.8,1.5])

ax.set_xlim([datetime.datetime(1905,6,1),datetime.datetime(2015,6,1)])


ax.set_ylim([-50,50])
ax.grid(linestyle='-', linewidth=.5)

ax.set_xlim([datetime.datetime(1905,6,1),datetime.datetime(2015,6,1)])


fig.text(0.5, -0.04, 'Year', ha='center')
fig.text(0.09, 0.5, '3yr running JO duration at 4,5,6', va='center', rotation='vertical')
ax.axvspan(datetime.datetime(1921,6,1),datetime.datetime(1945,6,1),color = 'r',alpha = 0.1)
ax.axvspan(datetime.datetime(1945,6,1),datetime.datetime(1977,6,1),color = 'b',alpha = 0.1)
ax.axvspan(datetime.datetime(1977,6,1),datetime.datetime(1999,6,1),color = 'r',alpha = 0.1)# ax.axvspan(1998,2012,alpha = 0.1)
ax.axvspan(datetime.datetime(1999,6,1),datetime.datetime(2012,6,1),color = 'b',alpha = 0.1)

plt.legend(ncol=4)
plt.title('MJO duration at 4,5,6')
ax = plt.gca()
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=60))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
# plt.gcf().autofmt_xdate() # Rotation
plt.show()


In [ ]:
##########################################

In [ ]:
plt.plot(df_conv_3yr.values[0:])
res = stats.linregress(np.arange(1,len(df_conv_3yr.values[0:])+1),df_conv_3yr.values.squeeze()[0:])
plt.plot(res[0]*np.arange(1,len(df_conv_3yr.values)+1)+res[1])
print(res)
plt.plot(df_mlr_3yr.values[0:])
res = stats.linregress(np.arange(1,len(df_mlr_3yr.values[0:])+1),df_mlr_3yr.values.squeeze()[0:])
plt.plot(res[0]*np.arange(1,len(df_mlr_3yr.values)+1)+res[1])
res

## connection with PDO

In [ ]:
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA
import matplotlib.pyplot as plt


plt.rcParams['font.weight'] = 'normal'
plt.rcParams['font.size'] = 20
plt.rcParams['image.cmap'] = 'seismic'

host = host_subplot(111, axes_class=AA.Axes)

# plt.plot(np.arange(1905,2015,1),t567_ol)

t567_conv_detrend = signal.detrend(t567_conv_hist)
plt.plot(np.arange(1905,2015,1),t567_conv_hist-np.mean(t567_conv_hist),'b',alpha=0.5)

# t567_ol_detrend = signal.detrend(t567_ol_hist)
# plt.plot(np.arange(1905,2015,1),t567_ol_hist-np.mean(t567_ol_hist),'r',alpha=0.3)


plt.plot(np.arange(1905,2015,1),lowpass_scipy_butter(t567_conv_hist-np.mean(t567_conv_hist),wn,10),'--b')
# plt.plot(np.arange(1905,2015,1),lowpass_scipy_butter(t567_ol_detrend,wn,10),'r')
mean_1 = 0

plt.axhline(mean_1,color ='k')
plt.ylim([mean_1-40,mean_1+40])



ax2 = host.twinx()
# ax3 = host.twinx()
offset = 80
# new_fixed_axis = ax3.get_grid_helper().new_fixed_axis
# ax3.axis["right"] = new_fixed_axis(loc="right",
#                                     axes=ax3,
#                                     offset=(offset, 0))

ax2.axis["right"].toggle(all=True)

offset = 20
new_fixed_axis = ax2.get_grid_helper().new_fixed_axis
ax2.axis["right"] = new_fixed_axis(loc="right",
                                    axes=ax2,
                                    offset=(offset, 0))

# ax3.axis["right"].toggle(all=True)

# ax2.bar(np.arange(1905,2015,1),pdot.values[:,0],color='k',alpha=0.3)
# ax2.plot(np.arange(1905,2015,1),lowpass_scipy_butter(pdot.values[:,0],wn,10),'--k',alpha=0.8)



pdo_time_series= pdo_winter_mean*1#pdot.values.squeeze()*1
ax2.bar(np.arange(1905,2015,1),pdo_time_series-np.mean(pdo_time_series),color='k',alpha=0.3)
ax2.plot(np.arange(1905,2015,1),lowpass_scipy_butter(pdo_time_series-np.mean(pdo_time_series),wn,10),'--k',alpha=0.8)


mean_2 =0
ax2.axhline(mean_2,color ='k')
ax2.set_ylim([mean_2-2,mean_2+2])

ax2.axvspan(1921,1944,color = 'r',alpha = 0.1)
ax2.axvspan(1944,1976,alpha = 0.1)
ax2.axvspan(1976,1997,color = 'r',alpha = 0.1)
ax2.axvspan(1997,2012,alpha = 0.1)


# ax3.plot(np.arange(1905,2015-35,1)+18,re_mlr,'orange')


# ax3.plot(np.arange(1905,2015-35,1)+k//2,re_lsq,'r')
# ax3.scatter(np.arange(1905,2015-35,1)+k//2,pp,color = 'r')

# ax3.set_ylim([-0.6,0.6])

host.axis["left"].label.set_color('k')
ax2.axis["right"].label.set_color('blue')
# ax3.axis["right"].label.set_color('r')



In [ ]:
plt.plot(t567_conv_hist)
plt.plot(t567_c2d_hist)

In [ ]:
start = 20
# print(np.corrcoef(pdot.values[start:,0],t567_conv_hist[start:]))
# print(np.corrcoef(pdot.values[start:,0],t567_ol_hist[start:]))


print(np.corrcoef(pdo_winter_mean[start:],t567_conv_hist[start:]))
print(np.corrcoef(pdo_winter_mean[start:],t567_ol_hist[start:]))
print(np.corrcoef(pdo_winter_mean[start:],t567_mlr_hist[start:]))

print(np.corrcoef(smooth(pdo_winter_mean[start:],window_len=7,window='flat')[3:-3],smooth(t567_conv_hist[start:],window_len=7,window='flat')[3:-3]))
print(np.corrcoef(smooth(pdo_winter_mean[start:],window_len=7,window='flat')[3:-3],smooth(t567_ol_hist[start:],window_len=7,window='flat')[3:-3]))
print(np.corrcoef(smooth(pdo_winter_mean[start:],window_len=7,window='flat')[3:-3],smooth(t567_mlr_hist[start:],window_len=7,window='flat')[3:-3]))

# print(np.corrcoef(lowpass_scipy_butter(pdot.values[start:,0],wn,10),lowpass_scipy_butter(t567_conv_hist[start:],wn,10)))
# print(np.corrcoef(lowpass_scipy_butter(pdot.values[start:,0],wn,10),lowpass_scipy_butter(t567_ol_hist[start:],wn,10)))

print(np.corrcoef(lowpass_scipy_butter(pdo_winter_mean[start:],wn,10),lowpass_scipy_butter(t567_conv_hist[start:],wn,10)))
print(np.corrcoef(lowpass_scipy_butter(pdo_winter_mean[start:],wn,10),lowpass_scipy_butter(t567_ol_hist[start:],wn,10)))
print(np.corrcoef(lowpass_scipy_butter(pdo_winter_mean[start:],wn,10),lowpass_scipy_butter(t567_mlr_hist[start:],wn,10)))

plt.figure(figsize=(8,8))
# plt.scatter(lowpass_scipy_butter(pdot.values[start:,0],wn,10),lowpass_scipy_butter(t567_conv_hist[start:],wn,10))
yy = t567_conv_hist[start:]*1
plt.scatter(pdo_winter_mean[start:],yy)
res = stats.linregress(pdo_winter_mean[start:],yy)
plt.plot(np.arange(-2,2,0.1),np.arange(-2,2,0.1)*res[0]+res[1])


plt.ylim([0,80])
plt.xlim([-2.5,2.5])
# plt.axvline(0,color='k')
# plt.axhline(45,color='k')

plt.xlabel('PDO')
plt.ylabel('duration456')
plt.title('correlation=%f'%res[2])

In [ ]:
import pandas as pd
from scipy import stats
plt.figure(figsize = (16,8))
# plt.plot(signal.detrend(t567_conv_hist),'r')
t567_10yr = smooth(t567_conv_hist,window_len=11,window='flat')[5:-5]
plt.plot(t567_10yr,'orange')
plt.plot(t567_conv_hist,'b')
res = stats.linregress(np.arange(1,len(t567_10yr[0:])+1), t567_10yr[0:])
plt.plot(res[0]*np.arange(1,len(t567_10yr)+1)+res[1])
res

In [ ]:
k= 35
re=np.zeros((len(t567_conv_hist)-k))
re_lsq=np.zeros((len(t567_conv_hist)-k))
pp = np.zeros((len(t567_conv_hist)-k))
pp.fill(np.nan)
re_mlr=np.zeros((len(t567_conv_hist)-k))
re_lsq_mlr=np.zeros((len(t567_conv_hist)-k))
pp_mlr = np.zeros((len(t567_conv_hist)-k))
pp_mlr.fill(np.nan)
yy=np.zeros((len(t567_conv_hist)-k))

year = np.arange(1905,2015)

for i in np.arange(len(t567_conv_hist)-k):
    x= np.arange(1,k+1)
    y= t567_conv_hist[i:i+k]
    y1= t567_ol_hist[i:i+k]
    
    
    res = stats.theilslopes(y, x)
    lsq_res = stats.linregress(x, y)
    re[i]=res[0]
    re_lsq[i]=lsq_res[0]
    if lsq_res[3]<0.05:
        pp[i] = res[0]
    
    
    res = stats.theilslopes(y1, x)
    lsq_res = stats.linregress(x, y1)
    re_mlr[i]=res[0]
    re_lsq_mlr[i]=lsq_res[0]
    yy[i] = year[i]
    if lsq_res[3]<0.05:
        pp_mlr[i] = res[0]

In [ ]:

plt.plot(yy+k//2,re_mlr,'b')
plt.scatter(yy+k//2,pp_mlr)

plt.plot(yy+k//2,re,'orange')
plt.scatter(yy+k//2,pp)
plt.axhline(0)

In [ ]:
%store -z

In [ ]:
%store df_567_all
%store df_567_all_3yr
%store amp1_wh04_filt 
%store phase1_wh04_filt


In [ ]:
# plt.figure(figsize=(10,10))
# plt.scatter(t567_svr_hist,t567_ol_hist)

In [ ]:
t567_c2d_hist,df_567_all.iloc[:,5]

## winter amplitude pdf 

In [ ]:
st1 = 1921; en1  = 1944 
st2 = 1945; en2  = 1976
st3 = 1977; en3  = 1997
st4 = 1998; en4  = 2012 

In [ ]:
AA= pd.DataFrame(mjo_ind_conv,phase1_conv_hist)
## conv1 ## 
a_conv1 = mjo_ind_conv_hist.loc[str(st1)+'-01-01':str(en1)+'-12-31']
a_conv2 = mjo_ind_conv_hist.loc[str(st2)+'-01-01':str(en2)+'-12-31']
a_conv3 = mjo_ind_conv_hist.loc[str(st3)+'-01-01':str(en3)+'-12-31']
a_conv4 = mjo_ind_conv_hist.loc[str(st4)+'-01-01':str(en4)+'-12-31']

a_conv1 = a_conv1[a_conv1.index.month.isin([1,2,3,4,11,12])]
a_conv2 = a_conv2[a_conv2.index.month.isin([1,2,3,4,11,12])]
a_conv3 = a_conv3[a_conv3.index.month.isin([1,2,3,4,11,12])]
a_conv4 = a_conv4[a_conv4.index.month.isin([1,2,3,4,11,12])]

In [ ]:
bins= np.arange(0,5,0.5)

n1_1,x1_1,_=plt.hist(a_conv1,bins=bins,density=True,alpha=0.5,histtype='step',label='observed')
n1_2,x1_2,_=plt.hist(a_conv2,bins=bins,density=True,alpha=0.5,histtype='step',label='observed')
n1_3,x1_3,_=plt.hist(a_conv3,bins=bins,density=True,alpha=0.5,histtype='step',label='ol')
n1_4,x1_4,_=plt.hist(a_conv4,bins=bins,density=True,alpha=0.5,histtype='step',label='observed')

n2_1,x2_1,_=plt.hist(a_conv1,bins=bins,density=True,alpha=0.5,cumulative= True,histtype='step',label='observed')
n2_2,x2_2,_=plt.hist(a_conv2,bins=bins,density=True,alpha=0.5,cumulative= True,histtype='step',label='observed')
n2_3,x2_3,_=plt.hist(a_conv3,bins=bins,density=True,alpha=0.5,cumulative= True,histtype='step',label='ol')
n2_4,x2_4,_=plt.hist(a_conv4,bins=bins,density=True,alpha=0.5,cumulative= True,histtype='step',label='observed')

plt.clf()

fig,ax=plt.subplots(1,2,figsize=(20,8))
plt.rcParams['font.weight'] = 'normal'
plt.rcParams['font.size'] = 15
plt.rcParams['image.cmap'] = 'seismic'

bin_centers = 0.5*(x1_1[1:]+x1_1[:-1])
ax[0].plot(bin_centers,n1_1,'-.',label='1921-1944')
ax[0].plot(bin_centers,n1_2,'-.',color='orange',label='1945-1976')
ax[0].plot(bin_centers,n1_3,'-.k',label='1977-1997')
ax[0].plot(bin_centers,n1_4,'-.',color='r',label='1998-2012')


ax[1].plot(bin_centers,n2_1,'-.',label='1921-1944')
ax[1].plot(bin_centers,n2_2,'-.',color='orange',label='1945-1976')
ax[1].plot(bin_centers,n2_3,'-.k',label='1977-1997')
ax[1].plot(bin_centers,n2_4,'-.',color='r',label='1998-2012')

plt.legend()
# bin_centers3 = 0.5*(x3[1:]+x3[:-1])
# plt.plot(bin_centers3,n3,'b',label='oliver')
# plt.legend(bbox_to_anchor=(0.6,-0.38),loc="lower right",ncol=5)
fig.text(0.5, -0.04, 'Amplitude', ha='center')
fig.text(0.04, 0.5, 'Percentage(%)', va='center', rotation='vertical')

## winter amplitude over 4,5,6 region 

In [ ]:
AA= pd.DataFrame([amp1_conv_hist,phase1_conv_hist]).T
AA.columns = ['amplitude','phase']
BB=AA.loc[AA.index.month.isin([1,2,3,4,11,12])]
BB = BB[ (BB['phase']>3) &(BB['phase']<7) ]
a_conv11 = BB.loc[str(st1)+'-11-01':str(en1)+'-04-30']
a_conv22 = BB.loc[str(st2)+'-11-01':str(en2)+'-04-30']
a_conv33 = BB.loc[str(st3)+'-11-01':str(en3)+'-04-30']
a_conv44 = BB.loc[str(st4)+'-11-01':str(en4)+'-04-30']

In [ ]:
bins= np.arange(0,5,0.5)

n1_1,x1_1,_=plt.hist(a_conv11.amplitude,bins=bins,density=True,alpha=0.5,cumulative= True,histtype='step',label='observed')
n1_2,x1_2,_=plt.hist(a_conv22.amplitude,bins=bins,density=True,alpha=0.5,cumulative= True,histtype='step',label='observed')
n1_3,x1_3,_=plt.hist(a_conv33.amplitude,bins=bins,density=True,alpha=0.5,cumulative= True,histtype='step',label='ol')
n1_4,x1_4,_=plt.hist(a_conv44.amplitude,bins=bins,density=True,alpha=0.5,cumulative= True,histtype='step',label='observed')


n2_1,x1_1,_=plt.hist(a_conv11.amplitude,bins=bins,density=True,alpha=0.5,histtype='step',label='observed')
n2_2,x1_2,_=plt.hist(a_conv22.amplitude,bins=bins,density=True,alpha=0.5,histtype='step',label='observed')
n2_3,x1_3,_=plt.hist(a_conv33.amplitude,bins=bins,density=True,alpha=0.5,histtype='step',label='ol')
n2_4,x1_4,_=plt.hist(a_conv44.amplitude,bins=bins,density=True,alpha=0.5,histtype='step',label='observed')


plt.clf()

fig,ax=plt.subplots(1,2,figsize=(20,8))
plt.rcParams['font.weight'] = 'normal'
plt.rcParams['font.size'] = 15
plt.rcParams['image.cmap'] = 'seismic'

bin_centers = 0.5*(x1_1[1:]+x1_1[:-1])
ax[0].plot(bin_centers,n1_1,'-.',label='1921-1944')
ax[0].plot(bin_centers,n1_2,'-.',color='orange',label='1945-1976')
ax[0].plot(bin_centers,n1_3,'-.k',label='1977-1997')
ax[0].plot(bin_centers,n1_4,'-.',color='r',label='1998-2012')


ax[1].plot(bin_centers,n2_1,'-.',label='1921-1944')
ax[1].plot(bin_centers,n2_2,'-.',color='orange',label='1945-1976')
ax[1].plot(bin_centers,n2_3,'-.k',label='1977-1997')
ax[1].plot(bin_centers,n2_4,'-.',color='r',label='1998-2012')

plt.legend()
# bin_centers3 = 0.5*(x3[1:]+x3[:-1])
# plt.plot(bin_centers3,n3,'b',label='oliver')
# plt.legend(bbox_to_anchor=(0.6,-0.38),loc="lower right",ncol=5)
fig.text(0.5, -0.04, 'Amplitude', ha='center')
fig.text(0.04, 0.5, 'Percentage(%)', va='center', rotation='vertical')

In [ ]:
df_567_all